In [1]:
import xml.etree.ElementTree

dump_file = 'wikipedia_2000_dump.xml'
tree = xml.etree.ElementTree.parse(dump_file)
root = tree.getroot()

In [2]:
def extract_alpha_words(page):
    words = page.text.split()
    alpha_words = list(filter(lambda word: word.isalpha(), words))
    alpha_lower_words = list(map(lambda word: word.lower(), alpha_words))
    return alpha_lower_words

pages_words = []
for page in root:
    pages_words.append(extract_alpha_words(page))
pages_count = len(pages_words)
print('Pages count =', pages_count)

Pages count = 2000


In [3]:
word_dict = {}
word_count = 0
for page_words in pages_words:
    for word in page_words:
        if word not in word_dict:
            word_dict[word] = word_count
            word_count += 1
print('Unique alpha words =', word_count)

Unique alpha words = 210492


In [4]:
import random

print("Random words' sample:")
words = list(word_dict.keys())
sample = random.sample(range(len(words)), 10)
for index in sample:
    print(words[index])

Random words' sample:
бризовой
инкоси
русское
прорвался
поддержавшим
голланца
градусам
первозданной
далекие
мексиканец


In [5]:
import numpy as np

X = np.zeros((pages_count, word_count), dtype=np.float32)
for index, page_words in enumerate(pages_words):
    for word in page_words:
        X[index][word_dict[word]] += (1 / len(page_words))

In [6]:
import math

def compute_erank(X):
    N, M = X.shape
    print('N = {}, M = {}'.format(N, M))
    C = X.dot(X.T)
    print('cov_matrix shape is {}'.format(C.shape))
    _, s, _ = np.linalg.svd(C)
    s_sum = np.sum(s)
    p = np.zeros(N)
    for i in range(N):
        p[i] = s[i] / s_sum
    entropy = 0
    for i in range(N):
        entropy -= p[i] * math.log(p[i])
    print('Entropy is = {}'.format(entropy))
    erank = math.exp(entropy)
    return erank

print('Effective rank =', compute_erank(X))

N = 2000, M = 210492
cov_matrix shape is (2000, 2000)
Entropy is = 4.495633504928133
Effective rank = 89.62492883852647
